In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import ipywidgets as widgets

pd.set_option("display.max_rows", 1000)

exp_path = Path("../experiments/issue_32/")
tex_path = exp_path / "tex_files_openbmi"
tex_path.mkdir(exist_ok=True)

In [2]:
!ls /home/paulo/Documents/GIT/BCI_MsC/experiments/issue_32/results_openbmi/

1   13	17  20	24  28	34  38	41  45	49  52	7	       results_cat.csv
10  14	18  21	25  29	35  39	42  46	5   53	8	       results.csv
11  15	19  22	26  3	36  4	43  47	50  54	9	       results_more.csv
12  16	2   23	27  30	37  40	44  48	51  6	results_2.csv  results_pc.csv


In [3]:
df = pd.read_csv("/home/paulo/Documents/GIT/BCI_MsC/experiments/issue_32/results_openbmi/results_cat.csv")
uids = df.uid.unique()

full_df = df[~df.algorithm.isin(["pca", "whitening"])]
# full_df = pd.read_csv("/home/paulo/Documents/GIT/BCI_MsC/experiments/issue_19/results.csv")

In [4]:
full_df.run.unique(), full_df.uid.unique(), full_df.uid.nunique()

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24,
        25, 26, 27, 28, 29,  3,  9,  8,  7,  6, 54, 52, 51, 50,  5, 49, 48,
        47, 46, 45, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,  4, 40, 41, 42,
        43, 44, 53]),
 54)

In [5]:
full_df.algorithm.unique(), full_df.classifier.unique()

(array(['None', 'ORICA (0)', 'ORICA (1)', 'Ext. Infomax', 'Infomax',
        'SOBI', 'JADE', 'Picard', 'FastICA', 'picard_o'], dtype=object),
 array(['mlp', 'random_forest', 'extra_trees', 'gaussian_nb', 'lda',
        'svm_sigmoid', 'svm_poly', 'svm_linear', 'svm_rbf', 'logistic_l2',
        'logistic_l1', 'logistic'], dtype=object))

In [6]:
def get_default(get_fn):
    def fn(key):
        return get_fn(key, key)
    return fn

classifier_rename_dict = dict(
    mlp="MLP",
    random_forest="Random Forest",
    extra_trees="Extra Trees",
    gaussian_nb="Naïve Bayes",
    lda="LDA",
    svm_sigmoid="SVM (Sig.)",
    svm_poly="SVM (Poly)",
    svm_linear="SVM (Lin.)",
    svm_rbf="SVM (RBF)",
    logistic_l1="Log. Reg. (L1)",
    logistic_l2="Log. Reg. (L2)",
    logistic="Log. Reg.",
)
algorithm_rename_dict = dict(
    picard_o="Picard-O",
)
full_df["algorithm"] = full_df.algorithm.apply(get_default(algorithm_rename_dict.get))
full_df["classifier"] = full_df.classifier.apply(get_default(classifier_rename_dict.get)) 

In [7]:
full_df.algorithm.unique(), full_df.classifier.unique()

(array(['None', 'ORICA (0)', 'ORICA (1)', 'Ext. Infomax', 'Infomax',
        'SOBI', 'JADE', 'Picard', 'FastICA', 'Picard-O'], dtype=object),
 array(['MLP', 'Random Forest', 'Extra Trees', 'Naïve Bayes', 'LDA',
        'SVM (Sig.)', 'SVM (Poly)', 'SVM (Lin.)', 'SVM (RBF)',
        'Log. Reg. (L2)', 'Log. Reg. (L1)', 'Log. Reg.'], dtype=object))

In [8]:
full_df.to_csv("formatted.csv", index=None)
!rm -rf ../experiments/issue_32/plots_openbmi
!python ../ica_benchmark/visualization/multiple_runs.py -path ./formatted.csv -save_folder ../experiments/issue_32/plots_openbmi

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

...

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/paulo/miniconda3/envs/bci/lib/python3.9/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal app

In [9]:
print(full_df.shape)
subset = set(full_df.columns) - set(["run"])
full_df = full_df.drop_duplicates(subset=subset)
print(full_df.shape)
full_df.columns

(64800, 11)
(60477, 11)


Index(['run', 'algorithm', 'classifier', 'uid', 'Acc.', 'Bal. Acc.', 'Kappa',
       'clf_fit_time', 'preprocess_fit_time', 'selected_features',
       'duplicated'],
      dtype='object')

In [10]:
df = full_df.groupby(["uid", "classifier", "algorithm"], as_index=False).mean()

# Best combination

In [28]:
sorted_all = full_df.groupby(["uid", "classifier", "algorithm"], as_index=False).agg([np.mean, np.std]).reset_index()#[["uid", "Kappa"]]
sorted_all["order"] = sorted_all.groupby("uid").rank(ascending=False)[("Kappa", "mean")]
sorted_all
ranked_df_list = list()
for uid in sorted_all.uid.unique():
    uid_df = sorted_all[sorted_all.uid == uid]
    max_rank = uid_df.order.min()
    r_df = uid_df[uid_df.order == max_rank][["uid", "classifier", "algorithm", "Kappa"]].round(3)
    ranked_df_list.append(r_df)
ranked_df = pd.concat(ranked_df_list).sort_values(by="uid")
ranked_df

/tmp/ipykernel_58866/3455617520.py:1: FutureWarning: ['selected_features'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  sorted_all = full_df.groupby(["uid", "classifier", "algorithm"], as_index=False).agg([np.mean, np.std]).reset_index()#[["uid", "Kappa"]]
/tmp/ipykernel_58866/3455617520.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  sorted_all["order"] = sorted_all.groupby("uid").rank(ascending=False)[("Kappa", "mean")]


uid      classifier     algorithm  Kappa       
                                         mean    std
40     1  Log. Reg. (L2)  Ext. Infomax  0.385  0.034
154    2  Log. Reg. (L1)          None  0.220  0.000
164    2  Log. Reg. (L2)          None  0.220  0.000
224    2       SVM (RBF)          None  0.220  0.000
271    3  Log. Reg. (L1)       FastICA  0.718  0.158
426    4     Naïve Bayes     ORICA (1)  0.190  0.000
534    5             MLP          None  0.482  0.024
627    6       Log. Reg.        Picard  0.634  0.041
628    6       Log. Reg.      Picard-O  0.634  0.041
742    7       Log. Reg.       Infomax  0.300  0.095
924    8      SVM (Lin.)          None  0.100  0.000
1014   9             MLP          None  0.383  0.059
1184  10       SVM (RBF)          None  0.150  0.000
1286  11      SVM (Lin.)     ORICA (1)  0.070  0.000
1296  11      SVM (Poly)     ORICA (1)  0.070  0.000
1419  12      SVM (Poly)          SOBI  0.180  0.000
1544  13       SVM (RBF)          None  0.170  0.000
1646  14      SVM (Lin.)     ORICA (1)  0.160  0.000
1666  14       SVM (RBF)     ORICA (1)  0.160  0.000
1745  15     Naïve Bayes     ORICA (0)  0.100  0.000
1815  16             LDA     ORICA (0)  0.250  0.000
1991  17   Random Forest       FastICA  0.291  0.057
2139  18      SVM (Poly)          SOBI  0.600  0.000
2279  19      SVM (Sig.)          SOBI  0.340  0.000
2289  20     Extra Trees          SOBI  0.133  0.042
2430  21  Log. Reg. (L1)  Ext. Infomax  0.954  0.020
2639  22      SVM (Sig.)          SOBI  0.450  0.000
2662  23       Log. Reg.       Infomax  0.333  0.063
2875  24      SVM (Sig.)     ORICA (0)  0.090  0.000
2904  25       Log. Reg.          None  0.340  0.000
3006  26     Extra Trees     ORICA (1)  0.110  0.071
3219  27      SVM (Poly)          SOBI  0.150  0.000
3279  28  Log. Reg. (L1)          SOBI  0.760  0.000
3384  29       Log. Reg.          None  0.970  0.000
3404  29  Log. Reg. (L2)          None  0.970  0.000
3444  29      SVM (Lin.)          None  0.970  0.000
3534  30             MLP          None  0.198  0.029
3700  31       SVM (RBF)  Ext. Infomax  0.111  0.075
3799  32   Random Forest          SOBI  0.751  0.029
3939  33      SVM (Poly)          SOBI  0.850  0.000
4066  34       SVM (RBF)     ORICA (1)  0.150  0.000
4130  35             MLP  Ext. Infomax  0.371  0.218
4220  36       Log. Reg.  Ext. Infomax  0.951  0.007
4260  36     Naïve Bayes  Ext. Infomax  0.951  0.010
4404  37      SVM (Lin.)          None  0.840  0.000
4506  38     Naïve Bayes     ORICA (1)  0.090  0.000
4612  39             MLP       Infomax  0.476  0.127
4796  40      SVM (Sig.)     ORICA (1)  0.100  0.000
4818  41             LDA      Picard-O  0.063  0.043
4817  41             LDA        Picard  0.063  0.043
4991  42   Random Forest       FastICA  0.173  0.077
5119  43   Random Forest          SOBI  0.311  0.014
5189  44       Log. Reg.          SOBI  0.960  0.000
5352  45   Random Forest       Infomax  0.804  0.030
5429  46       Log. Reg.          SOBI  0.450  0.000
5449  46  Log. Reg. (L2)          SOBI  0.450  0.000
5624  47       SVM (RBF)          None  0.110  0.000
5719  48   Random Forest          SOBI  0.198  0.085
5874  49      SVM (Sig.)          None  0.340  0.000
5975  50      SVM (Poly)     ORICA (0)  0.140  0.000
6111  51      SVM (Sig.)       FastICA  0.283  0.024
6189  52     Naïve Bayes          SOBI  0.330  0.000
6304  53     Naïve Bayes          None  0.100  0.000
6344  53       SVM (RBF)          None  0.100  0.000
6455  54      SVM (Poly)     ORICA (0)  0.180  0.000

In [12]:
ranked_df.to_latex(tex_path / "best_combination.tex", index=False)

/tmp/ipykernel_58866/4233368133.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  ranked_df.to_latex(tex_path / "best_combination.tex", index=False)


# Best ICA's

In [13]:
sorted_alg = df.groupby(["uid", "algorithm"], as_index=False).agg([np.mean, np.std]).Kappa.sort_values(by=["uid", "mean"]).reset_index()
sorted_alg["order"] = sorted_alg.groupby("uid").rank(ascending=False)["mean"]
sorted_alg

/tmp/ipykernel_58866/2109070944.py:1: FutureWarning: ['classifier'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  sorted_alg = df.groupby(["uid", "algorithm"], as_index=False).agg([np.mean, np.std]).Kappa.sort_values(by=["uid", "mean"]).reset_index()


,uid,algorithm,mean,std,order
0,1,ORICA (0),-0.080833,0.031389,10.0
1,1,JADE,0.013750,0.022547,9.0
2,1,ORICA (1),0.022500,0.025816,8.0
3,1,None,0.177667,0.061614,7.0
4,1,Picard,0.252500,0.011572,5.5
5,1,Picard-O,0.252500,0.011572,5.5
6,1,SOBI,0.256417,0.041903,4.0
7,1,Infomax,0.257750,0.011733,3.0
8,1,FastICA,0.276500,0.015067,2.0
9,1,Ext. Infomax,0.364750,0.017131,1.0


In [14]:
ranked_df_list = list()
for uid in sorted_alg.uid.unique():
    uid_df = sorted_alg.query("uid == @uid")
    max_rank = uid_df.order.min()
    r_df = uid_df.query("order == @max_rank")[["uid", "algorithm", "mean", "std"]].round(3)
    ranked_df_list.append(r_df)
ranked_df = pd.concat(ranked_df_list).sort_values(by="uid")
ranked_df

,uid,algorithm,mean,std
9,1,Ext. Infomax,0.365,0.017
19,2,None,0.169,0.050
29,3,None,0.640,0.035
39,4,ORICA (0),0.119,0.019
49,5,None,0.395,0.118
58,6,Picard,0.596,0.031
59,6,Picard-O,0.596,0.031
69,7,Infomax,0.267,0.035
79,8,None,0.043,0.030
89,9,None,0.332,0.069


In [15]:
ranked_df.to_latex(tex_path / "best_alg.tex", index=False)

/tmp/ipykernel_58866/1770953152.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  ranked_df.to_latex(tex_path / "best_alg.tex", index=False)


In [16]:
order_df = sorted_alg.groupby("algorithm").median()[["order"]].sort_values(by="order")
order_df

,order
algorithm,
Ext. Infomax,4.5
Picard,4.5
Picard-O,4.5
FastICA,5.0
Infomax,5.0
None,6.0
SOBI,6.0
ORICA (0),8.0
JADE,8.5


In [17]:
order_df.to_latex(tex_path / "best_alg_median_rank.tex", index=False)

/tmp/ipykernel_58866/2213213182.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  order_df.to_latex(tex_path / "best_alg_median_rank.tex", index=False)


# Best classifier

In [18]:
sorted_clf = df.groupby(["uid", "classifier"], as_index=False).agg([np.mean, np.std]).Kappa.sort_values(by=["uid", "mean"]).reset_index()
sorted_clf["order"] = sorted_clf.groupby("uid").rank(ascending=False)["mean"]
ranked_df_list = list()
for uid in sorted_clf.uid.unique():
    uid_df = sorted_clf.query("uid == @uid")
    max_rank = uid_df.order.min()
    r_df = uid_df.query("order == @max_rank")[["uid", "classifier", "mean", "std"]].round(3)
    ranked_df_list.append(r_df)
ranked_df = pd.concat(ranked_df_list).sort_values(by="uid")
ranked_df

/tmp/ipykernel_58866/3632222478.py:1: FutureWarning: ['algorithm'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  sorted_clf = df.groupby(["uid", "classifier"], as_index=False).agg([np.mean, np.std]).Kappa.sort_values(by=["uid", "mean"]).reset_index()


,uid,classifier,mean,std
11,1,SVM (Lin.),0.193,0.137
23,2,SVM (Lin.),0.095,0.098
35,3,Extra Trees,0.572,0.146
47,4,Random Forest,0.088,0.043
58,5,LDA,0.197,0.110
59,5,Log. Reg.,0.197,0.117
71,6,Log. Reg. (L1),0.520,0.125
83,7,Naïve Bayes,0.165,0.137
95,8,SVM (Lin.),0.010,0.041
107,9,Random Forest,0.206,0.167


In [19]:
ranked_df.to_latex(tex_path / "best_clf.tex", index=False)

/tmp/ipykernel_58866/3553676854.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  ranked_df.to_latex(tex_path / "best_clf.tex", index=False)


In [20]:
order_df = sorted_clf.groupby("classifier").median()[["order"]].sort_values(by="order")
order_df

,order
classifier,
Log. Reg. (L1),4.0
Log. Reg.,5.0
Log. Reg. (L2),5.0
SVM (Lin.),5.0
Random Forest,6.0
SVM (Sig.),6.5
LDA,7.0
MLP,7.0
Extra Trees,8.0


In [21]:
order_df.to_latex(tex_path / "best_clf_median_rank.tex", index=False)

/tmp/ipykernel_58866/4039510481.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  order_df.to_latex(tex_path / "best_clf_median_rank.tex", index=False)


In [22]:
save_folder = Path("../experiments/issue_32/plots_openbmi/")

classifiers = full_df.classifier.unique()
tab_contents = classifiers
children = list()
for clf in classifiers:
    with open(save_folder / f"best_algorithm_per_subject_for_{clf}.png", "rb") as f:
        widget = widgets.Image(
            value=f.read(),
            format='png',
        )
    children.append(widget)

tab = widgets.Tab()
tab.children = children
for i, clf in enumerate(classifiers):
    tab.set_title(i, clf)
tab

In [23]:
algorithms = full_df.algorithm.unique()
tab_contents = algorithms
children = list()
for alg in algorithms:
    with open(save_folder / f"best_classifier_per_subject_for_{alg}.png", "rb") as f:
        widget = widgets.Image(
            value=f.read(),
            format='png',
        )
    children.append(widget)

tab = widgets.Tab()
tab.children = children
for i, clf in enumerate(algorithms):
    tab.set_title(i, alg)
tab

In [24]:
classifiers = full_df.classifier.unique()
tab_contents = classifiers
children = list()
for clf in classifiers:
    with open(save_folder / f"average_per_algorithm_for_{clf}.png", "rb") as f:
        widget = widgets.Image(
            value=f.read(),
            format='png',
        )
    children.append(widget)

tab = widgets.Tab()
tab.children = children
for i, clf in enumerate(classifiers):
    tab.set_title(i, clf)
tab

In [25]:
algorithms = full_df.algorithm.unique()
tab_contents = algorithms
children = list()
for alg in algorithms:
    with open(save_folder / f"average_per_classifier_for_{alg}.png", "rb") as f:
        widget = widgets.Image(
            value=f.read(),
            format='png',
        )
    children.append(widget)

tab = widgets.Tab()
tab.children = children
for i, clf in enumerate(algorithms):
    tab.set_title(i, alg)
tab

In [26]:
classifiers = full_df.classifier.unique()
tab_contents = classifiers
children = list()
for clf in classifiers:
    with open(save_folder / f"detailed_{clf}.png", "rb") as f:
        widget = widgets.Image(
            value=f.read(),
            format='png',
        )
    children.append(widget)

tab = widgets.Tab()
tab.children = children
for i, clf in enumerate(classifiers):
    tab.set_title(i, clf)
tab

In [27]:
algorithms = full_df.algorithm.unique()
tab_contents = algorithms
children = list()
for alg in algorithms:
    with open(save_folder / f"detailed_{alg}.png", "rb") as f:
        widget = widgets.Image(
            value=f.read(),
            format='png',
        )
    children.append(widget)

tab = widgets.Tab()
tab.children = children
for i, clf in enumerate(algorithms):
    tab.set_title(i, clf)
tab